In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import os
import sys
from cvxpy import *
from multiprocessing import Pool
from functools import partial
from numpy import linalg as LA
from tqdm.notebook import tqdm
from scipy.sparse import coo_array
from scipy.optimize import nnls
import multiprocessing as mp
import pickle as pkl
import numpy as np
import pandas as pd
import scipy as sp
import math
import time

# Non-spatial TopicScore

In [ ]:
def unpack_apply(all_args):
    (func1d, axis, arr) = all_args
    return np.apply_along_axis(func1d, axis, arr)


def solveX_LS(data):
    n, p, k = data[(data.size - 3) : (data.size)].astype(int)
    d = data[0:p]
    x_sol = nnls(A.transpose(), d)[0]
    return x_sol

In [ ]:
# Load Ahat, D, E
root_path = ""
sample_idx = ["BALBc-1", "BALBc-2", "BALBc-3"]
PATH_TO_MODELS = "spleen_tscore_nonspatial"
n_topics = [3, 5, 8, 10]
lamb = 0

for sample in sample_idx:
    print(f"Sample {sample} starting...")
    print("---------------------------------")
    # Read D
    D_path = f"{sample}_D_unnorm" + ".csv"
    D = pd.read_csv(root_path + D_path)
    names = D.to_numpy()[:, 0]
    D = D.to_numpy()[:, 1:]
    D = D / D.sum(axis=1, keepdims=True)  # normalize to fraction
    # Iterate for each n_topic
    for n_topic in n_topics:
        print(f"Running n_topics={n_topic}, lambda={lamb}\n")
        t = time.time()
        path_to_train_model = (
            "_".join(
                (
                    f"{PATH_TO_MODELS}",
                    f"{sample}" f"penalty={lamb}",
                    f"topics={n_topic}",
                )
            )
            + ".pkl"
        )
        # Read Ahat
        Ahat_path = f"{sample}_Ahat_" + f"{n_topic}" + ".csv"
        Ahat = pd.read_csv(root_path + Ahat_path)
        A = Ahat.to_numpy().transpose()
        n, p = D.shape
        k = A.shape[0]
        # Run
        data = np.concatenate(
            (D.transpose(), np.tile([n, p, k], (n, 1)).transpose()), axis=0
        )
        chunks = [
            (solveX_LS, 1, sub_arr)
            for sub_arr in np.array_split(data.transpose(), mp.cpu_count())
        ]
        pool = Pool()
        values = pool.map(unpack_apply, chunks)
        res = np.concatenate(values)
        pool.close()
        pool.join()
        x = res.transpose()  # k*n
        # edge = E[:,:-1].astype(int)
        # edge_expand = np.zeros(2*m)
        # edge_expand[::2] = edge[:,0]
        # edge_expand[1::2] = edge[:,1]
        # z = x[:,edge_expand.astype(int)] #k*2m
        # u = np.zeros((k,2*m))
        elapsed = time.time() - t
        print(f"Elapsed time={elapsed}\n")
        with open(path_to_train_model, "wb") as f:
            pkl.dump(x, f)

# Spatial TopicScore

## ADMM functions

In [ ]:
def solveX(data, vars, edge_mat):
    rho, lamb = data[(data.size - 6) : (data.size - 4)]
    m, n, p, k = data[(data.size - 4) : (data.size)].astype(int)
    nodenum = data[(data.size - 7)]
    x = data[0:k]
    d = data[k : (k + p)]
    Z_k = np.concatenate(
        (
            vars[(edge_mat[:, 0] == nodenum), :k],
            vars[(edge_mat[:, 1] == nodenum), k : 2 * k],
        ),
        axis=0,
    )
    U_k = np.concatenate(
        (
            vars[(edge_mat[:, 0] == nodenum), 2 * k : 3 * k],
            vars[(edge_mat[:, 1] == nodenum), 3 * k : 4 * k],
        ),
        axis=0,
    )
    xnew = Variable(k)

    g = 0.5 * square(norm(d - (xnew @ A)))
    V = U_k - Z_k + vstack([xnew] * Z_k.shape[0])  # x - z + u
    h = 0.5 * rho * sum(square(norm(V, axis=1)))
    objective = Minimize(g + h)
    constraints = []  # constraints = [xnew >= 0]
    pr = Problem(objective, constraints)
    result = pr.solve(solver="ECOS_BB")

    if result == None:
        objective = Minimize(51 * g + 52 * h)
        pr = Problem(objective, constraints)
        result = pr.solve(verbose=False, warm_start=True)
        if result == None:
            print("SCALING BUG")  # CVXOPT scaling issue
            objective = Minimize(52 * g + 50 * h)
            pr = Problem(objective, constraints)
            result = pr.solve(verbose=False, warm_start=True)
    return xnew.value


def solveX_wrapper(args):
    row, vars, edge = args
    return solveX(row, vars=vars, edge=edge)


def solveX_ext(data, vars, edge_val):
    rho, lamb = data[(data.size - 6) : (data.size - 4)]
    m, n, p, k = data[(data.size - 4) : (data.size)].astype(int)
    nodenum = data[(data.size - 7)]
    x = data[0:k]
    d = data[k : (k + p)]
    Z_k = np.concatenate(
        (
            vars[(edge_val[:, 0] == nodenum), :k],
            vars[(edge_val[:, 1] == nodenum), k : 2 * k],
        ),
        axis=0,
    )
    U_k = np.concatenate(
        (
            vars[(edge_val[:, 0] == nodenum), 2 * k : 3 * k],
            vars[(edge_val[:, 1] == nodenum), 3 * k : 4 * k],
        ),
        axis=0,
    )
    M = Z_k.shape[0]
    AAT_inv = LA.inv(A @ A.transpose() + rho * M * np.eye(k))
    B = (A @ d) + rho * np.sum(Z_k - U_k, axis=0)
    xnew = AAT_inv @ B
    return xnew


def solveX_est_apply(all_args):
    (func1d, axis, arr, vars, edge) = all_args
    return np.apply_along_axis(func1d, axis, arr, vars, edge)


def solveZ(data):
    rho, lamb = data[(data.size - 6) : (data.size - 4)]
    m, n, p, k = data[(data.size - 4) : (data.size)].astype(int)
    x1 = data[0:k]
    x2 = data[k : 2 * k]
    u1 = data[2 * k : 3 * k]
    u2 = data[3 * k : 4 * k]
    weight = data[data.size - 7]
    a = x1 + u1
    b = x2 + u2
    theta = np.max(
        np.array([1 - lamb * weight / (rho * LA.norm(a - b) + 0.000001), 0.5])
    )
    z1 = theta * a + (1 - theta) * b
    z2 = theta * b + (1 - theta) * a
    znew = np.matrix(np.concatenate([z1, z2])).reshape(2 * k, 1)
    return znew


def solveU(data):
    k = int(data[data.size - 1])
    u = data[0:k]
    x = data[k : 2 * k]
    z = data[2 * k : 3 * k]
    return u + (x - z)

In [ ]:
def runADMM(d, A, E, P, x, u, z, rho, lamb, m, n, p, k, numiters):
    # Stopping criteria
    eabs = math.pow(10, -3)
    erel = math.pow(10, -4)
    (r, s, epri, edual) = (1, 1, 0, 0)  # E = np.zeros((2*m, n))
    (sqn, sqp) = (math.sqrt(n * k), math.sqrt(2 * k * m))

    # E for dual residual
    edge = E[:, :-1].astype(int)
    weights = 1 / E[:, -1]
    weights = (weights - np.min(weights)) / (np.max(weights) - np.min(weights))

    # Run ADMM
    iters = 0
    while iters < numiters and (r > epri or s > edual or iters < 1):
        # print(f'Iter{iters} starting...')
        # x-update
        zu = np.concatenate(
            (
                z.reshape(2 * k, m, order="F").transpose(),
                u.reshape(2 * k, m, order="F").transpose(),
            ),
            axis=1,
        )
        temp = np.concatenate(
            (
                x,
                D.transpose(),
                np.arange(n).reshape((1, n)),
                np.tile([rho, lamb, m, n, p, k], (n, 1)).transpose(),
            ),
            axis=0,
        )
        chunks = [
            (solveX_ext, 1, sub_arr, zu, edge)
            for sub_arr in np.array_split(temp.transpose(), mp.cpu_count())
        ]
        pool = Pool()
        newx = pool.map(solveX_est_apply, chunks)
        pool.close()
        pool.join()
        xnew = np.concatenate(newx)
        x = np.array(xnew.tolist()).transpose()  # k x n
        # newcvxObj = np.array(values)[:,1].tolist()
        # cvxObj = np.reshape(np.array(newcvxObj), (-1, n))

        # z-update
        ztemp = z.reshape(2 * k, m, order="F")
        utemp = u.reshape(2 * k, m, order="F")
        xtemp = np.concatenate((x[:, edge[:, 0]], x[:, edge[:, 1]]), axis=0)  # 2k x m
        temp = np.concatenate(
            (
                xtemp,
                utemp,
                weights.reshape((1, m)),
                np.tile([rho, lamb, m, n, p, k], (m, 1)).transpose(),
            ),
            axis=0,
        )
        chunks = [
            (solveZ, 1, sub_arr)
            for sub_arr in np.array_split(temp.transpose(), mp.cpu_count())
        ]
        pool = Pool()
        newz = pool.map(unpack_apply, chunks)
        pool.close()
        pool.join()
        znew = np.concatenate(newz)
        ztemp = np.array(znew.tolist()).transpose()
        ztemp = ztemp.reshape(k, 2 * m, order="F")  # k x 2m
        ztemp = np.where(ztemp < 0, 0, ztemp)  # nonnegative projection
        s = LA.norm(
            rho * P.transpose().dot((ztemp - z).transpose())
        )  # For dual residual
        z = ztemp

        # u-update
        xtemp = xtemp.reshape(k, 2 * m, order="F")
        unew = u + xtemp - z
        u = unew  # k x 2m

        # Stopping criterion - p19 of ADMM paper
        epri = sqp * eabs + erel * np.max(
            np.array(
                [LA.norm(x, "fro"), LA.norm(P.transpose().dot(z.transpose()), "fro")]
            )
        )
        edual = sqn * eabs + erel * LA.norm(u.transpose(), "fro")
        # edual = sqn*eabs + erel*LA.norm(P.transpose().dot(u.transpose()), 'fro')
        r = LA.norm(x.transpose() - P.transpose().dot(z.transpose()), "fro")
        s = s

        # print r, epri, s, edual
        iters = iters + 1
    print(f"{iters} iterations were run.")
    return (x, u, z, 0, 0)

## Check CPU cores

In [ ]:
import multiprocessing as mp

num_cores = mp.cpu_count()
print(num_cores)

4


## RUN

In [ ]:
# Load Ahat, D, E
root_path = ""
sample_idx = ["BALBc-1", "BALBc-2", "BALBc-3"]
PATH_TO_MODELS = "spleen_tscore"
n_topics = [3, 5, 8, 10]
lamb = 4  # spatial penalty
rho = 0.1  # ADMM parameter
numiters = 50  # Max number of ADMM iterations

for sample in sample_idx:
    print(f"Sample {sample} starting...")
    print("---------------------------------")
    # Read D and edge array
    D_path = f"{sample}_D_unnorm" + ".csv"
    E_path = f"{sample}_edge" + ".csv"
    D = pd.read_csv(root_path + D_path)
    names = D.to_numpy()[:, 0]
    D = D.to_numpy()[:, 1:]
    D = D / D.sum(axis=1, keepdims=True)  # normalize to fraction
    E = pd.read_csv(root_path + E_path)
    E = E.to_numpy()

    # Create sparse transformation matrix P (2*m,n)
    # x = zP
    m = E.shape[0]
    mat = np.zeros((2 * m, 2))
    mat[:, 0] = np.arange(0, 2 * m)
    mat[::2, 1] = E[:, 0]
    mat[1::2, 1] = E[:, 1]
    row = mat[:, 0]
    col = mat[:, 1]
    data = np.ones(2 * m)
    P_unnorm = coo_array((data, (row.astype(int), col.astype(int)))).tocsr()
    count = np.array(P_unnorm.sum(axis=0)).flatten()
    P = P_unnorm.multiply(1 / count)  # 2m*n
    # np.sum(P.transpose().dot(z.transpose())-x.transpose()) # should be near 0

    # Iterate for each n_topic
    for n_topic in n_topics:
        print(f"Running n_topics={n_topic}, lambda={lamb}\n")
        t = time.time()
        path_to_train_model = (
            "_".join(
                (
                    f"{PATH_TO_MODELS}",
                    f"{sample}",
                    f"penalty={lamb}",
                    f"topics={n_topic}",
                )
            )
            + ".pkl"
        )
        # Read Ahat
        Ahat_path = f"{sample}_Ahat_" + f"{n_topic}" + ".csv"
        Ahat = pd.read_csv(root_path + Ahat_path)
        A = Ahat.to_numpy().transpose()
        n, p = D.shape
        k = A.shape[0]
        # Set initial parameters
        x = np.zeros((k, n))
        z = np.zeros((k, 2 * m))
        u = np.zeros((k, 2 * m))
        # Run
        rhoval = np.min(np.array([lamb + 0.00001, rho + lamb / 25]))
        (x, u, z, pl1, pl2) = runADMM(
            D, A, E, P, x, u, z, rhoval, lamb, m, n, p, k, numiters
        )
        elapsed = time.time() - t
        print(f"Elapsed time={elapsed}\n")
        with open(path_to_train_model, "wb") as f:
            pkl.dump(x, f)

Sample BALBc-1 starting...
---------------------------------
Running n_topics=3, lambda=4

16 iterations were run.
Elapsed time=1371.316017150879

Running n_topics=5, lambda=4

14 iterations were run.
Elapsed time=1570.261428117752

Running n_topics=8, lambda=4

10 iterations were run.
Elapsed time=1131.2875337600708

Running n_topics=10, lambda=4

9 iterations were run.
Elapsed time=1044.0989921092987

Sample BALBc-2 starting...
---------------------------------
Running n_topics=3, lambda=4

15 iterations were run.
Elapsed time=1201.2846179008484

Running n_topics=5, lambda=4

12 iterations were run.
Elapsed time=1244.5845758914948

Running n_topics=8, lambda=4

10 iterations were run.
Elapsed time=1040.165302991867

Running n_topics=10, lambda=4

10 iterations were run.
Elapsed time=1053.6275823116302

Sample BALBc-3 starting...
---------------------------------
Running n_topics=3, lambda=4

14 iterations were run.
Elapsed time=1018.6702148914337

Running n_topics=5, lambda=4

10 ite

In [ ]:
# from google.colab import files
# files.download('no_spatial_11')